In [82]:
import pandas as pd
import numpy as np

In [69]:
df_input = pd.read_excel('./Sequenciamentos/Base Algoritmo - RJ 26.08.2022 (CENÁRIO 01 - FINAL).xlsx', sheet_name='Obras')
df_de_para = pd.read_excel('./Sequenciamentos/Base Algoritmo - RJ 26.08.2022 (CENÁRIO 01 - FINAL).xlsx', sheet_name='Setup')
df_custos_depara = pd.read_excel('./Sequenciamentos/SKUs.xlsx', sheet_name = 'custos')

In [70]:
def verifica_overlap(tarefa1, tarefa2):
    inicio1 = pd.to_datetime(tarefa1['Inicio_obra'])
    fim1 = pd.to_datetime(tarefa1['Fim_obra'])
    inicio2 = pd.to_datetime(tarefa2['Inicio_obra'])
    fim2 = pd.to_datetime(tarefa2['Fim_obra'])
    
    # Verifica se há overlap temporal
    if inicio1 <= fim2 and inicio2 <= fim1:
        return 'Overlap'
    else:
        return 'Sem overlap'

# Criação do novo dataframe com a verificação de overlap
df_setup = pd.DataFrame(columns=['De', 'Para', 'Verificação'])

# Itera sobre cada par de tarefas do dataframe original
for i in range(len(df_input)):
    for j in range(len(df_input)):
        tarefa1 = df_input.iloc[i]
        tarefa2 = df_input.iloc[j]
        resultado = verifica_overlap(tarefa1, tarefa2)
        
        # Adiciona o resultado ao novo dataframe
        df_setup = df_setup.append({
            'De': tarefa1['N_VERTICE'],
            'Para': tarefa2['N_VERTICE'],
            'Verificação': resultado
        }, ignore_index=True)

# Exibe o novo dataframe
df_setup['Custo Setup'] = 0
df_setup

,De,Para,Verificação,Custo Setup
0,0,0,Overlap,0
1,0,1,Sem overlap,0
2,0,2,Overlap,0
3,0,3,Sem overlap,0
4,0,4,Sem overlap,0
...,...,...,...,...
524,22,18,Overlap,0
525,22,19,Overlap,0
526,22,20,Overlap,0
527,22,21,Sem overlap,0


In [71]:
df_custos_depara.rename(columns={'Forma/Obra' : 'Index'}, inplace = True)
df_custos_depara.set_index('Index', inplace = True)
df_custos_depara.columns = map(str.upper, df_custos_depara.columns)
df_custos_depara.index = map(str.upper, df_custos_depara.index)

In [73]:
BIG_M = 100000000
for i in range(len(df_setup['De'])):
    if df_setup.loc[i, 'Verificação'] == 'Overlap':
        df_setup['Custo Setup'][i] = BIG_M
    elif  df_setup.loc[i, 'Verificação'] != 'Overlap' and df_input.loc[df_setup['De'][i],'Modelo Necessário'] != 'coringa' and df_input.loc[df_setup['Para'][i],'Modelo Necessário'] != 'coringa':
        df_setup['Custo Setup'][i] = df_custos_depara.loc[df_input.loc[df_setup['De'][i],'Modelo Necessário'],df_input.loc[df_setup['Para'][i],'Modelo Necessário']]
df_setup

C:\Users\luis.hreis\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
C:\Users\luis.hreis\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,De,Para,Verificação,Custo Setup
0,0,0,Overlap,100000000
1,0,1,Sem overlap,2682510
2,0,2,Overlap,100000000
3,0,3,Sem overlap,2682510
4,0,4,Sem overlap,2682510
...,...,...,...,...
524,22,18,Overlap,100000000
525,22,19,Overlap,100000000
526,22,20,Overlap,100000000
527,22,21,Sem overlap,63501


In [92]:
txt = ''
for index in range(len(df_setup)):
    vrt_1 = df_setup.loc[index, 'De']
    vrt_2 = df_setup.loc[index, 'Para']
    if df_setup.loc[index, 'De'] != df_setup.loc[index, 'Para']:
        weight = df_setup.loc[index, 'Custo Setup']
        txt += f'e {vrt_1} {vrt_2}\n'
txt

'e 0 1\ne 0 2\ne 0 3\ne 0 4\ne 0 5\ne 0 6\ne 0 7\ne 0 8\ne 0 9\ne 0 10\ne 0 11\ne 0 12\ne 0 13\ne 0 14\ne 0 15\ne 0 16\ne 0 17\ne 0 18\ne 0 19\ne 0 20\ne 0 21\ne 0 22\ne 1 0\ne 1 2\ne 1 3\ne 1 4\ne 1 5\ne 1 6\ne 1 7\ne 1 8\ne 1 9\ne 1 10\ne 1 11\ne 1 12\ne 1 13\ne 1 14\ne 1 15\ne 1 16\ne 1 17\ne 1 18\ne 1 19\ne 1 20\ne 1 21\ne 1 22\ne 2 0\ne 2 1\ne 2 3\ne 2 4\ne 2 5\ne 2 6\ne 2 7\ne 2 8\ne 2 9\ne 2 10\ne 2 11\ne 2 12\ne 2 13\ne 2 14\ne 2 15\ne 2 16\ne 2 17\ne 2 18\ne 2 19\ne 2 20\ne 2 21\ne 2 22\ne 3 0\ne 3 1\ne 3 2\ne 3 4\ne 3 5\ne 3 6\ne 3 7\ne 3 8\ne 3 9\ne 3 10\ne 3 11\ne 3 12\ne 3 13\ne 3 14\ne 3 15\ne 3 16\ne 3 17\ne 3 18\ne 3 19\ne 3 20\ne 3 21\ne 3 22\ne 4 0\ne 4 1\ne 4 2\ne 4 3\ne 4 5\ne 4 6\ne 4 7\ne 4 8\ne 4 9\ne 4 10\ne 4 11\ne 4 12\ne 4 13\ne 4 14\ne 4 15\ne 4 16\ne 4 17\ne 4 18\ne 4 19\ne 4 20\ne 4 21\ne 4 22\ne 5 0\ne 5 1\ne 5 2\ne 5 3\ne 5 4\ne 5 6\ne 5 7\ne 5 8\ne 5 9\ne 5 10\ne 5 11\ne 5 12\ne 5 13\ne 5 14\ne 5 15\ne 5 16\ne 5 17\ne 5 18\ne 5 19\ne 5 20\ne 5 21\ne 5 2

In [94]:
with open('graph.txt', 'w') as f:
    f.write(txt)

In [83]:
ndarray_custo_setup = df_setup.pivot(index='De', columns='Para', values='Custo Setup').to_numpy()
np.fill_diagonal(ndarray_custo_setup, 0)

In [85]:
adj_matrix = ndarray_custo_setup

In [91]:
np.savetxt('adj_matrix.txt', adj_matrix)